In [ ]:
from google.colab import files
uploaded = files.upload()


Saving DEMO_L.XPT to DEMO_L.XPT
Saving HEPB_S_L.XPT to HEPB_S_L.XPT
Saving KIQ_U_L.XPT to KIQ_U_L.XPT
Saving PAQ_L.XPT to PAQ_L.XPT
Saving VID_L.XPT to VID_L.XPT
Saving BAX_L.XPT to BAX_L.XPT
Saving BPXO_L.XPT to BPXO_L.XPT


In [ ]:

import pandas as pd

# Load the data
demo_data = pd.read_sas('DEMO_L.XPT', format='xport')

hepb_data = pd.read_sas('HEPB_S_L.XPT', format='xport')

kidney_data = pd.read_sas('KIQ_U_L.XPT', format='xport')

paq_data = pd.read_sas('PAQ_L.XPT', format='xport')

vid_data = pd.read_sas('VID_L.XPT', format='xport')

bax_data = pd.read_sas('BAX_L.XPT', format='xport')

bp_data = pd.read_sas('BPXO_L.XPT', format='xport')



In [ ]:
# Preview first few rows of each dataset
print(demo_data.head())
print(hepb_data.head())
print(kidney_data.head())
print(paq_data.head())
print(vid_data.head())
print(bax_data.head())
print(bp_data.head())


       SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDRETH1  \
0  130378.0      12.0       2.0       1.0      43.0       NaN       5.0   
1  130379.0      12.0       2.0       1.0      66.0       NaN       3.0   
2  130380.0      12.0       2.0       2.0      44.0       NaN       2.0   
3  130381.0      12.0       2.0       2.0       5.0       NaN       5.0   
4  130382.0      12.0       2.0       1.0       2.0       NaN       3.0   

   RIDRETH3  RIDEXMON  RIDEXAGM  ...  DMDHRGND  DMDHRAGZ  DMDHREDZ  DMDHRMAZ  \
0       6.0       2.0       NaN  ...       NaN       NaN       NaN       NaN   
1       3.0       2.0       NaN  ...       NaN       NaN       NaN       NaN   
2       2.0       1.0       NaN  ...       NaN       NaN       NaN       NaN   
3       7.0       1.0      71.0  ...       2.0       2.0       2.0       3.0   
4       3.0       2.0      34.0  ...       2.0       2.0       3.0       1.0   

   DMDHSEDZ      WTINT2YR      WTMEC2YR  SDMVSTRA  SDMVPSU  INDFMPIR

In [ ]:
# Merge datasets based on SEQN
merged_data = demo_data.merge(hepb_data, on='SEQN', how='inner') \
                       .merge(kidney_data, on='SEQN', how='inner') \
                       .merge(paq_data, on='SEQN', how='inner') \
                       .merge(vid_data, on='SEQN', how='inner') \
                       .merge(bax_data, on='SEQN', how='inner') \
                       .merge(bp_data, on='SEQN', how='inner')


In [ ]:
# Remove placeholder values for PAD680 and WHD020
merged_data = merged_data[(merged_data['PAD680'] != 7777) & (merged_data['PAD680'] != 9999)]

# Drop rows with null values in continuous variables
merged_data = merged_data.dropna(subset=['PAD680'])


In [ ]:
#Question 1: "Is there an association between marital status (married or not married) and education level (bachelor’s degree or higher vs. less than a bachelor’s degree)?"
import pandas as pd
from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(merged_data['DMDMARTZ_recode'], merged_data['DMDEDUC2_recode'])

# chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Output results
print("Chi-Square Statistic:", chi2)
print("p-value:", p)

# Interpretation
if p < 0.05:
    print("There is a statistically significant association between marital status and education level.")
else:
    print("There is no statistically significant association between marital status and education level.")


Chi-Square Statistic: 70.77972596712145
p-value: 3.9942066155967015e-17
There is a statistically significant association between marital status and education level.


In [ ]:
from scipy.stats import ttest_ind

# Separate data into two groups: married and not married
married_group = merged_data[merged_data['DMDMARTZ_recode'] == 1]['PAD680']
not_married_group = merged_data[merged_data['DMDMARTZ_recode'] == 0]['PAD680']

# t-test
t_stat, p_value = ttest_ind(married_group, not_married_group)

print("t-statistic:", t_stat)
print("p-value:", p_value)

# Interpretation
if p_value < 0.05:
    print("There is a statistically significant difference in mean sedentary behavior time between married and not married individuals.")
else:
    print("There is no statistically significant difference in mean sedentary behavior time between married and not married individuals.")


t-statistic: -1.8261644570177828
p-value: 0.06788838069584785
There is no statistically significant difference in mean sedentary behavior time between married and not married individuals.


In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Recode Marital Status and Education Level
merged_data['DMDMARTZ_recode'] = merged_data['DMDMARTZ'].apply(lambda x: 1 if x == 1 else 0)
merged_data['DMDEDUC2_recode'] = merged_data['DMDEDUC2'].apply(lambda x: 1 if x >= 5 else 0)

# Create a contingency table
contingency_table = pd.crosstab(merged_data['DMDMARTZ_recode'], merged_data['DMDEDUC2_recode'])

# chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Display results
print("Chi-Square Statistic:", chi2)
print("p-value:", p)

# Interpretation
if p < 0.05:
    print("There is a statistically significant association between marital status and education level.")
else:
    print("There is no statistically significant association between marital status and education level.")


Chi-Square Statistic: 70.77972596712145
p-value: 3.9942066155967015e-17
There is a statistically significant association between marital status and education level.
